In [2]:
import torch
import numpy as np

# Create a tensor first
tensor = torch.randn(3, 4)  # Example tensor with random values

if torch.cuda.is_available():
  tensor = tensor.to('cuda')
  print(f"Device tensor is stored on: {tensor.device}")
else:
  print("error")

error


In [3]:
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\camil/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:03<00:00, 13.7MB/s]


In [ ]:
prediction = model(data) # forward pass

loss_initial = (prediction - labels).sum()
loss_initial.backward() # backward pass

optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

optim.step() #gradient descent

tensor(-5070.8271, grad_fn=<SumBackward0>)
tensor(-5070.8271, grad_fn=<SumBackward0>)


In [1]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init()

    self.conv1 = nn.Conv2d(1, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)

    self.fc1 = nn.Linear(16*5*5, 120 )
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  
  def forward(self, input):
    c1 = F.relu(self.conv1)
    s2 = F.max_pool2d(c1, (2,2))
    c3 = F.relu(self.conv2(s2))
    s4 = F.max_pool2d(c3, 2)
    # Flatten operation: purely functional, outputs a (N, 400) Tensor
    s4 = torch.flatten(s4, 1)
    # Fully connected layer F5: (N, 400) Tensor input,
    # and outputs a (N, 120) Tensor, it uses RELU activation function
    f5 = F.relu(self.fc1(s4))
    # Fully connected layer F6: (N, 120) Tensor input,
    # and outputs a (N, 84) Tensor, it uses RELU activation function
    f6 = F.relu(self.fc2(f5))
    # Gaussian layer OUTPUT: (N, 84) Tensor input, and
    # outputs a (N, 10) Tensor
    output = self.fc3(f6)
    return output


In [1]:
import torch
from torch.utils.data import Dataset
import pandas as pd

class BlinkDataset(Dataset):
    def __init__(self, csv_file):
        df = pd.read_csv(csv_file)
        # Drop rows with missing values (optional)
        df = df.dropna()
        # Features: ratio, ratio_avg, distance_vertical, distance_horizontal
        self.X = df[['ratio', 'ratio_avg', 'distance_vertical', 'distance_horizontal']].values.astype('float32')
        # Labels: manual_blink (0 or 1)
        self.y = df['manual_blink'].values.astype('int64')
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])

In [2]:
import torch.nn as nn

class BlinkNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 2)  # 2 outputs: blink or not blink
        )
    def forward(self, x):
        return self.net(x)